In [1]:
%pip install simpy

In [2]:
# Vamos a simular un productor/consumidor genérico.
# El productor deja algo en el Store (que tiene una determinada capacidad)
# y el consumidor los toma.

# En este caso, hay un productor que deja objetos cada 2 segundos y dos
# consumidores que lo intentan tomar cada 1 segundo. Por lo tanto, los que
# esperan son los consumidores

import simpy

def productor(env, store):
  for i in range(100):
    yield env.timeout(2)
    print(f'Productor dejando objeto {i} a las {env.now}')
    yield store.put(f'objeto {i}')
    print(f'Productor dejó objeto {i} a las {env.now}')

def consumidor(obj, env, store):
  while True:
    yield env.timeout(1)
    print(f'Consumidor {obj} buscando objeto a las {env.now}')
    item = yield store.get()
    print(f'Consumidor {obj} tomó {item} a las {env.now}')

env = simpy.Environment()
store = simpy.Store(env, capacity=2)
prod = env.process(productor(env, store))
cons = [env.process(consumidor(i, env, store)) for i in range(2)]
env.run(until=8)


Consumidor 0 buscando objeto a las 1
Consumidor 1 buscando objeto a las 1
Productor dejando objeto 0 a las 2
Productor dejó objeto 0 a las 2
Consumidor 0 tomó objeto 0 a las 2
Consumidor 0 buscando objeto a las 3
Productor dejando objeto 1 a las 4
Productor dejó objeto 1 a las 4
Consumidor 1 tomó objeto 1 a las 4
Consumidor 1 buscando objeto a las 5
Productor dejando objeto 2 a las 6
Productor dejó objeto 2 a las 6
Consumidor 0 tomó objeto 2 a las 6
Consumidor 0 buscando objeto a las 7


In [5]:
# En este caso, hay un productor que deja objetos cada 1 segundo y dos
# consumidores que lo intentan tomar cada 5 segundos.

#Por lo tanto, los que esperan son los productores ya que el store tiene capacidad limitada

import simpy

def productor(env, store):
  for i in range(100):
    yield env.timeout(1)
    print(f'Productor dejando objeto {i} a las {env.now}')
    yield store.put(f'objeto {i}')
    print(f'Productor dejó objeto {i} a las {env.now}')

def consumidor(obj, env, store):
  while True:
    yield env.timeout(5)
    print(f'Consumidor {obj} buscando objeto a las {env.now}')
    item = yield store.get()
    print(f'Consumidor {obj} tomó {item} a las {env.now}')

env = simpy.Environment()
store = simpy.Store(env, capacity=2)
prod = env.process(productor(env, store))
cons = [env.process(consumidor(i, env, store)) for i in range(2)]
env.run(until=11)



Productor dejando objeto 0 a las 1
Productor dejó objeto 0 a las 1
Productor dejando objeto 1 a las 2
Productor dejó objeto 1 a las 2
Productor dejando objeto 2 a las 3
Consumidor 0 buscando objeto a las 5
Consumidor 1 buscando objeto a las 5
Consumidor 0 tomó objeto 0 a las 5
Consumidor 1 tomó objeto 1 a las 5
Productor dejó objeto 2 a las 5
Productor dejando objeto 3 a las 6
Productor dejó objeto 3 a las 6
Productor dejando objeto 4 a las 7
Consumidor 0 buscando objeto a las 10
Consumidor 1 buscando objeto a las 10
Consumidor 0 tomó objeto 2 a las 10
Consumidor 1 tomó objeto 3 a las 10
Productor dejó objeto 4 a las 10


In [6]:
# Otro ejemplo. Envío de mensajes (emisor->receptor)

# Se envian mensajes por la red que son procesados por el receptor luego de una
# determinada latencia.

import simpy

TIEMPO_SIMULACION = 100

class Cable(object):
    def __init__(self, env, delay):
        self.env = env
        self.delay = delay
        self.store = simpy.Store(env)

    def latencia(self, nombre):
        yield self.env.timeout(self.delay)
        self.store.put(nombre)

    def put(self, nombre):
        self.env.process(self.latencia(nombre))

    def get(self):
        return self.store.get()


def emisor(env, cable):
    while True:
        yield env.timeout(5)
        cable.put(f'Emisor lo envió a las {env.now}')


def receptor(env, cable):
    while True:
        msg = yield cable.get()
        print(f'Recibido a las {env.now} mientras que el {msg}')


env = simpy.Environment()

cable = Cable(env, 10)
env.process(emisor(env, cable))
env.process(receptor(env, cable))

env.run(until=TIEMPO_SIMULACION)

Recibido a las 15 mientras que el Emisor lo envió a las 5
Recibido a las 20 mientras que el Emisor lo envió a las 10
Recibido a las 25 mientras que el Emisor lo envió a las 15
Recibido a las 30 mientras que el Emisor lo envió a las 20
Recibido a las 35 mientras que el Emisor lo envió a las 25
Recibido a las 40 mientras que el Emisor lo envió a las 30
Recibido a las 45 mientras que el Emisor lo envió a las 35
Recibido a las 50 mientras que el Emisor lo envió a las 40
Recibido a las 55 mientras que el Emisor lo envió a las 45
Recibido a las 60 mientras que el Emisor lo envió a las 50
Recibido a las 65 mientras que el Emisor lo envió a las 55
Recibido a las 70 mientras que el Emisor lo envió a las 60
Recibido a las 75 mientras que el Emisor lo envió a las 65
Recibido a las 80 mientras que el Emisor lo envió a las 70
Recibido a las 85 mientras que el Emisor lo envió a las 75
Recibido a las 90 mientras que el Emisor lo envió a las 80
Recibido a las 95 mientras que el Emisor lo envió a las 8